In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_trf
nlp = en_core_web_trf.load()

import re

# # Import the wordcloud library
# from wordcloud import WordCloud

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
from pprint import pprint

# from gensim.models import CoherenceModel
# import tqdm

import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\duchi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#functions

def cleanText(text):
#     text = re.sub(r'@[A-Za-z0-9:_]+', '', text)  # remove @mensions
    text = re.sub(r'#', ' ', text)  # remove '#'
    text = re.sub(r'&amp;', ' ', text)
#     text = re.sub(r'RT[\s]+', '', text)  # remove RT
    text = re.sub(r'https?:\/\/\S+', ' ', text)  # remove the hyper link
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) # remove emoji
    text = re.sub(r'[.,"\'?:!;]', ' ', text)  # remove punctions
    text = re.sub(u"([^\u0041-\u005a\u0061-\u007a\u0020\u0027])", ' ', text)  # just keep words, spaces and single quote
    return text

def join_string(file):
    text = str()
    for line in file:
        #     print(line)
        if line.find('Word count') != -1:
            pass
        elif line.find('CREDIT:') != -1:
            pass
        else:
            text = text + " " + line.strip('\n ')

    return text

# join the proquest article text

In [3]:
#join the proquest article text

pq_link = pd.read_csv(
    r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\2 newspaper\proquest_link.csv',
    encoding='iso-8859-1')
pq_link.columns = ['link','company']

l1 = np.arange(1, 11)
l1 = [x * 100 for x in l1]
print(l1)
detail = pd.DataFrame()
for fix in l1:
    temp = pd.read_csv(
        r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\2 newspaper\news_{}.csv'
        .format(fix),
        encoding='iso-8859-1')
    detail = pd.concat([detail,temp], ignore_index=True)
    
pq_link = pd.merge(pq_link,detail, how='left', on=['link','company'])

l2 = []
for i in range(0, 983):
    
    file = open(
        r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\2 newspaper\proquest text\{}.txt'
        .format(i),
        encoding="utf8")
    
    l2.append(join_string(file))
    
pq_link['text'] = l2

[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]


In [4]:
# pq_link.to_excel(r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\3 summary\proquest_news tech summary.xlsx')

In [32]:
i = 516
print(pq_link['company'].loc[i])
text = pq_link['text'].loc[i]

doc = nlp(text)
filter = {'ents':['ORG','PERSON']}
displacy.render(doc, style="ent", jupyter=True, minify= True, options = filter)

Opentute


In [6]:
#clean the text
pq_link['processed_text'] = pq_link['text'].map(lambda x: cleanText(x))
pq_link['processed_text'] = pq_link['processed_text'].map(lambda x: x.lower())

long_string = ','.join(list(pq_link['processed_text'].values))

In [7]:
# #generate wordcloud
# wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# wordcloud.generate(long_string)
# wordcloud.to_image()

In [8]:
#some more function
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# data = pq_link['processed_text'].values.tolist()
# data_words = list(sent_to_words(data))

data = pq_link['processed_text'].loc[97]
data_words = list(gensim.utils.simple_preprocess(str(data), deacc=True))
# remove stop words
data_words_nostops = remove_stopwords(data_words)
print(data_words[:1][0][:30])

#for LDA analysis
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

the


In [9]:
# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# nlp = spacy.load("en_core_web_lg")
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[[]]


In [10]:
#Data Transformation: Corpus and Dictionary
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[]]


In [11]:
# # Build LDA model
# lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                        id2word=id2word,
#                                        num_topics=5, 
#                                        random_state=100,
#                                        chunksize=100,
#                                        passes=10,
#                                        per_word_topics=True,
#                                        workers=3)


# # Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [12]:
top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 100)])

lda_model_100 = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P'])

lda_model_100[['Topic','Word']].groupby('Topic')['Word'].unique().reset_index().to_csv('test.csv', index=None, encoding='utf-8')

NameError: name 'lda_model' is not defined

In [ ]:
# # Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

In [ ]:
# # supporting function
# def compute_coherence_values(corpus, dictionary, k, a, b):
    
#     lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                            id2word=dictionary,
#                                            num_topics=k, 
#                                            random_state=100,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha=a,
#                                            eta=b)
    
#     coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
#     return coherence_model_lda.get_coherence()

In [ ]:
# grid = {}
# grid['Validation_Set'] = {}
# # Topics range
# min_topics = 2
# max_topics = 11
# step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
# # Alpha parameter
# alpha = list(np.arange(0.01, 1, 0.3))
# alpha.append('symmetric')
# alpha.append('asymmetric')
# # Beta parameter
# beta = list(np.arange(0.01, 1, 0.3))
# beta.append('symmetric')
# # Validation sets
# num_of_docs = len(corpus)
# corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
#                # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#                gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
#                corpus]
# corpus_title = ['75% Corpus', '100% Corpus']
# model_results = {'Validation_Set': [],
#                  'Topics': [],
#                  'Alpha': [],
#                  'Beta': [],
#                  'Coherence': []
#                 }

In [ ]:
# # Can take a long time to run
# if 1 == 1:
#     pbar = tqdm.tqdm(total=540)
    
#     # iterate through validation corpuses
#     for i in range(len(corpus_sets)):
#         # iterate through number of topics
#         for k in topics_range:
#             # iterate through alpha values
#             for a in alpha:
#                 # iterare through beta values
#                 for b in beta:
#                     # get the coherence score for the given parameters
#                     cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
#                                                   k=k, a=a, b=b)
#                     # Save the model results
#                     model_results['Validation_Set'].append(corpus_title[i])
#                     model_results['Topics'].append(k)
#                     model_results['Alpha'].append(a)
#                     model_results['Beta'].append(b)
#                     model_results['Coherence'].append(cv)
                    
#                     pbar.update(1)
#     pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
#     pbar.close()

# model_results = pd.read_csv('lda_tuning_results.csv', encoding='utf-8')

In [ ]:
#Final Model

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=100, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9,
                                           workers=3)

In [ ]:
#Visualize Topics
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared